In [34]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
#from graphviz import Source
from sklearn import tree
#Source( tree.export_graphviz(dtreg, out_file=None, feature_names=X.columns))

In [2]:
df = pd.read_csv('wgm_2018_uk.csv')
df['id'] = df.index
df.head()

,WP5,wgt,PROJWT,FIELD_DATE,YEAR_CALENDAR,Q1,Q2,Q3,Q4,Q5A,...,AgeCategories,Gender,Education,Urban_Rural,Household_Income,Regions_Report,Subjective_Income,WBI,EMP_2010,id
0,12,0.660212,35676.829222,5/1/2018,2018,3,3,1,2,1,...,3,2,1,2,2,15,2,4,2,0
1,12,1.454006,78572.191694,5/1/2018,2018,1,1,2,2,1,...,2,1,2,2,2,15,2,4,1,1
2,12,0.305742,16521.821242,5/1/2018,2018,2,3,98,98,2,...,1,1,3,2,3,15,3,4,3,2
3,12,0.611130,33024.522413,5/1/2018,2018,4,2,1,2,1,...,2,1,3,2,5,15,1,4,1,3
4,12,0.793783,42894.795054,5/1/2018,2018,2,1,1,2,1,...,1,1,3,2,1,15,2,4,1,4


In [4]:
#1 = yes, 2 = no, 98 = dk, 99 = refused
#Q10B - confidence in hospitals etc
print(df.groupby('Q10B').wgt.sum())
#1 = a lot, 2 = some, 3 = not much, 4 = not at all, 98 = dk, 99 = refused
#How much do you trust...?
#Q11A neighbourhood
print(df.groupby('Q11A').wgt.sum())
#Q11B national government
print(df.groupby('Q11B').wgt.sum())
#Q11E doctors and nurses
print(df.groupby('Q11E').wgt.sum())
#Q11F NGOs
#print(df.groupby('Q11F').wgt.sum())

Q10B
1     824.678643
2     154.718155
98     20.279115
99      0.324087
Name: wgt, dtype: float64
Q11A
1     532.929008
2     346.997576
3      76.743026
4      29.956166
98     13.374224
Name: wgt, dtype: float64
Q11B
1     117.175653
2     393.052672
3     244.114663
4     229.566563
98     11.120957
99      4.969491
Name: wgt, dtype: float64
Q11E
1     659.692782
2     267.970447
3      50.502863
4      17.278988
98      3.098711
99      1.456209
Name: wgt, dtype: float64


In [5]:
#how much do you trust doctors/nurses for medical advice?
print(df.groupby('Q22').wgt.sum())
#Q20 - who do you trust most for health/medical advice? 1=family/friends, 2=religious leader, 3=doctor/nurse, 4=famous person, 5='traiditional healer', 97=none/other, 98=dk 99=refused
#print(df.groupby('Q20').wgt.count())
#1 = a lot, 2 = some, 3 = not much, 4 = not at all, 98 = dk, 99 = refused
#if doctor/nurse, how much do you trust their medical advice?
#print(df[df.Q20==3].groupby('Q22').wgt.count()/1515)
#if NOT doctor/nurse, how much do you trust their medical advice?
#print(df[df.Q20!=3].groupby('Q22').wgt.count()/485)

#Q20 - who do you trust most for health/medical advice? 1=family/friends, 2=religious leader, 3=doctor/nurse, 4=famous person, 5='traiditional healer', 97=none/other, 98=dk 99=refused
print(df.groupby('Q20').wgt.sum())
#1 = a lot, 2 = some, 3 = not much, 4 = not at all, 98 = dk, 99 = refused
#if doctor/nurse, how much do you trust their medical advice?
#print(df[df.Q20==3].groupby('Q22').wgt.sum()/1494.698709)
#if NOT doctor/nurse, how much do you trust their medical advice?
#print(df[df.Q20!=3].groupby('Q22').wgt.sum()/504.673230)
#print(df[df.Q20!=3].wgt.sum())

Q22
1     660.119115
2     262.599471
3      43.864555
4      27.894068
98      4.795789
99      0.727003
Name: wgt, dtype: float64
Q20
1      42.517786
2       6.306538
3     903.452807
4       8.333953
5      19.925747
97     15.634436
98      3.828733
Name: wgt, dtype: float64


In [6]:
df['med_trust'] = df.apply(lambda row: 0 if row['Q22'] > 4 else 4 - row['Q22'], axis=1)
df['med_trust_b'] = df.apply(lambda row: 1 if row['Q22'] < 3 else 0, axis=1)
df['pref_med'] = df.apply(lambda row: 1 if row['Q20'] == 3 else 0, axis=1)
df['med_trust_gen'] = df.apply(lambda row: 0 if row['Q11E'] > 4 else 4 - row['Q11E'], axis=1)
df['med_trust_gen_b'] = df.apply(lambda row: 1 if row['Q11E'] < 3 else 0, axis=1)
print(df.med_trust.mean())
print(df.med_trust_b.mean())
print(df.med_trust_b.sum())
print(df.pref_med.mean())
print(df.med_trust_gen.mean())
print(df.med_trust_gen_b.mean())

2.556
0.931
931
0.891
2.57
0.927


In [7]:
def trust_diff(a, b):
    if a<5 and b<5:
        return a - b
    else:
        return

In [8]:
print(df.groupby('Q21').wgt.sum())
print(df.groupby('Q22').wgt.sum())

print(df.groupby('Q11B').wgt.sum())
print(df.groupby('Q11E').wgt.sum())

df['gov_health_v_gen'] = df.apply(lambda row: trust_diff(row['Q21'], row['Q11B']), axis=1)
print(df.groupby('gov_health_v_gen').wgt.sum())
df['med_v_gov'] = df.apply(lambda row: trust_diff(row['Q22'], row['Q21']), axis=1)
print(df.groupby('med_v_gov').wgt.sum())
df['med_v_gov_gen'] = df.apply(lambda row: trust_diff(row['Q11E'], row['Q11B']), axis=1)
print(df.groupby('med_v_gov_gen').wgt.sum())

Q21
1     414.211265
2     400.621593
3     146.486878
4      29.347116
98      7.170035
99      2.163112
Name: wgt, dtype: float64
Q22
1     660.119115
2     262.599471
3      43.864555
4      27.894068
98      4.795789
99      0.727003
Name: wgt, dtype: float64
Q11B
1     117.175653
2     393.052672
3     244.114663
4     229.566563
98     11.120957
99      4.969491
Name: wgt, dtype: float64
Q11E
1     659.692782
2     267.970447
3      50.502863
4      17.278988
98      3.098711
99      1.456209
Name: wgt, dtype: float64
gov_health_v_gen
-3.0     62.165714
-2.0    168.256303
-1.0    359.288784
 0.0    307.725011
 1.0     67.346706
 2.0      8.710175
 3.0      2.248707
Name: wgt, dtype: float64
med_v_gov
-3.0     11.804851
-2.0     62.435967
-1.0    251.085302
 0.0    599.891896
 1.0     59.392868
 2.0      2.773656
Name: wgt, dtype: float64
med_v_gov_gen
-3.0    140.735954
-2.0    208.337673
-1.0    361.043738
 0.0    217.473654
 1.0     45.824328
 2.0      5.939286
Name: wgt, dtype

In [9]:
#trust by subjective income
print(df.groupby('Subjective_Income').wgt.sum())
print(df.groupby(['Subjective_Income', 'med_v_gov_gen']).wgt.sum()/df.groupby('Subjective_Income').wgt.sum())

#trust by employment status
print(df.groupby('EMP_2010').wgt.sum())
print(df.groupby(['EMP_2010', 'med_v_gov_gen']).wgt.sum()/df.groupby('EMP_2010').wgt.sum())

#trust by urban_rural
print(df.groupby('Urban_Rural').wgt.sum())
print(df.groupby(['Urban_Rural', 'med_v_gov_gen']).wgt.sum()/df.groupby('Urban_Rural').wgt.sum())

#trust by age cohort
print(df.groupby('AgeCategories').wgt.sum())
print(df.groupby(['AgeCategories', 'med_v_gov_gen']).wgt.sum()/df.groupby('AgeCategories').wgt.sum())

#trust by education
print(df.groupby('Education').wgt.sum())
print(df.groupby(['Education', 'med_v_gov_gen']).wgt.sum()/df.groupby('Education').wgt.sum())

#religion
print(df.groupby('D1').wgt.sum())
print(df.groupby(['D1', 'med_v_gov_gen']).wgt.sum()/df.groupby('D1').wgt.sum())

Subjective_Income
      12.289031
1    408.913177
2    417.738902
3    161.058891
Name: wgt, dtype: float64
Subjective_Income  med_v_gov_gen
                   -3.0             0.398282
                   -1.0             0.118497
                    0.0             0.288071
1                  -3.0             0.131231
                   -2.0             0.171164
                   -1.0             0.398251
                    0.0             0.244890
                    1.0             0.037833
                    2.0             0.005822
2                  -3.0             0.128252
                   -2.0             0.207629
                   -1.0             0.363986
                    0.0             0.215630
                    1.0             0.056019
                    2.0             0.003723
3                  -3.0             0.177597
                   -2.0             0.320452
                   -1.0             0.277453
                    0.0             0.147262
    

In [9]:
#independence of trust variables
#gov health and gov general

In [10]:
inc_trust = df.groupby(['Subjective_Income', 'Q11B']).wgt.sum().reset_index()
#print(inc_trust)
inc_trust = inc_trust.pivot(columns='Q11B', index='Subjective_Income', values='wgt')
inc_trust = inc_trust.iloc[[1,2,3],[0,1,2,3]]
#print(inc_trust)
stat, p, dof, ex = chi2_contingency(inc_trust)
print(p)

4.033679500375618e-06


In [11]:
inc_trust = (df.groupby(['Subjective_Income', 'Q11B']).wgt.sum()/df.groupby('Subjective_Income').wgt.sum()).reset_index()
inc_trust = inc_trust.pivot(columns='Q11B', index='Subjective_Income', values='wgt')
inc_trust = inc_trust.iloc[[1,2,3],[0,1,2,3]]
print(inc_trust)

Q11B                      1         2         3         4
Subjective_Income                                        
1                  0.147829  0.443724  0.227631  0.170007
2                  0.109419  0.384737  0.259009  0.227236
3                  0.057736  0.304653  0.247876  0.368106


In [14]:
inc_trust = df.groupby(['Subjective_Income', 'Q21']).wgt.sum().reset_index()
#print(inc_trust)
inc_trust = inc_trust.pivot(columns='Q21', index='Subjective_Income', values='wgt')
inc_trust = inc_trust.iloc[[1,2,3],[0,1,2,3]]
#print(inc_trust)
stat, p, dof, ex = chi2_contingency(inc_trust)
print(p)

1.8557781410199313e-07


In [15]:
inc_trust = (df.groupby(['Subjective_Income', 'Q21']).wgt.sum()/df.groupby('Subjective_Income').wgt.sum()).reset_index()
inc_trust = inc_trust.pivot(columns='Q21', index='Subjective_Income', values='wgt')
inc_trust = inc_trust.iloc[[1,2,3],[0,1,2,3]]
print(inc_trust)

Q21                       1         2         3         4
Subjective_Income                                        
1                  0.492078  0.380133  0.090487  0.029209
2                  0.400138  0.414273  0.158348  0.019950
3                  0.254850  0.427451  0.242898  0.056311


In [16]:
urb_trust = df.groupby(['Urban_Rural', 'Q11B']).wgt.sum().reset_index()
#print(urb_trust)
urb_trust = urb_trust.pivot(columns='Q11B', index='Urban_Rural', values='wgt')
#print(urb_trust)
urb_trust = urb_trust.iloc[[1,2],[0,1,2,3]]
#print(urb_trust)
stat, p, dof, ex = chi2_contingency(urb_trust)
print(p)

0.8890964169819993


In [17]:
urb_trust = df.groupby(['Urban_Rural', 'Q21']).wgt.sum().reset_index()
#print(urb_trust)
urb_trust = urb_trust.pivot(columns='Q21', index='Urban_Rural', values='wgt')
#print(urb_trust)
urb_trust = urb_trust.iloc[[1,2],[0,1,2,3]]
#print(urb_trust)
stat, p, dof, ex = chi2_contingency(urb_trust)
print(p)

0.7316684196913086


In [18]:
urb_trust = (df.groupby(['Urban_Rural', 'Q11B']).wgt.sum()/df.groupby('Urban_Rural').wgt.sum()).reset_index()
#print(urb_trust)
urb_trust = urb_trust.pivot(columns='Q11B', index='Urban_Rural', values='wgt')
#print(urb_trust)
urb_trust = urb_trust.iloc[[1,2],[0,1,2,3]]
print(urb_trust)

Q11B                1         2         3         4
Urban_Rural                                        
1            0.113908  0.404657  0.248120  0.222926
2            0.121978  0.380705  0.240484  0.233457


In [19]:
ed_trust = df.groupby(['Education', 'Q11B']).wgt.sum().reset_index()
#print(ed_trust)
ed_trust = ed_trust.pivot(columns='Q11B', index='Education', values='wgt')
ed_trust = ed_trust.iloc[[1,2,3],[0,1,2,3]]
#print(ed_trust)
stat, p, dof, ex = chi2_contingency(ed_trust)
print(p)

0.005010191297451153


In [20]:
ed_trust = df.groupby(['Education', 'Q21']).wgt.sum().reset_index()
#print(ed_trust)
ed_trust = ed_trust.pivot(columns='Q21', index='Education', values='wgt')
ed_trust = ed_trust.iloc[[1,2,3],[0,1,2,3]]
#print(ed_trust)
stat, p, dof, ex = chi2_contingency(ed_trust)
print(p)

0.42717200090150476


In [21]:
age_trust = df.groupby(['AgeCategories', 'Q11B']).wgt.sum().reset_index()
#print(age_trust)
age_trust = age_trust.pivot(columns='Q11B', index='AgeCategories', values='wgt')
age_trust = age_trust.iloc[[1,2,3],[0,1,2,3]]
#print(inc_trust)
stat, p, dof, ex = chi2_contingency(age_trust)
print(p)

0.00011406980149598103


In [22]:
age_trust = df.groupby(['AgeCategories', 'Q21']).wgt.sum().reset_index()
#print(age_trust)
age_trust = age_trust.pivot(columns='Q21', index='AgeCategories', values='wgt')
age_trust = age_trust.iloc[[1,2,3],[0,1,2,3]]
#print(inc_trust)
stat, p, dof, ex = chi2_contingency(age_trust)
print(p)

0.011676777552249768


In [23]:
age_trust = (df.groupby(['AgeCategories', 'Q11B']).wgt.sum()/df.groupby('AgeCategories').wgt.sum()).reset_index()
age_trust = age_trust.pivot(columns='Q11B', index='AgeCategories', values='wgt')
age_trust = age_trust.iloc[[1,2,3],[0,1,2,3]]
print(age_trust)

Q11B                  1         2         3         4
AgeCategories                                        
1              0.079362  0.458762  0.295548  0.152426
2              0.103003  0.400737  0.269304  0.212148
3              0.148540  0.352424  0.205324  0.275285


In [24]:
emp_trust = df.groupby(['EMP_2010', 'Q11B']).wgt.sum().reset_index()
emp_trust = emp_trust.pivot(columns='Q11B', index='EMP_2010', values='wgt')
#print(emp_trust)
emp_trust = emp_trust.iloc[[0,1,2,3,4,5],[0,1,2,3]]
#print(emp_trust)
stat, p, dof, ex = chi2_contingency(emp_trust)
print(p)

0.018287095267608678


In [25]:
emp_trust = df.groupby(['EMP_2010', 'Q21']).wgt.sum().reset_index()
emp_trust = emp_trust.pivot(columns='Q21', index='EMP_2010', values='wgt')
#print(emp_trust)
emp_trust = emp_trust.iloc[[0,1,2,3,4,5],[0,1,2,3]]
#print(emp_trust)
stat, p, dof, ex = chi2_contingency(emp_trust)
print(p)

4.1737248918632096e-05


In [26]:
emp_trust = (df.groupby(['EMP_2010', 'Q21']).wgt.sum()/df.groupby('EMP_2010').wgt.sum()).reset_index()
emp_trust = emp_trust.pivot(columns='Q21', index='EMP_2010', values='wgt')
emp_trust = emp_trust.iloc[[0,1,2,3,4,5],[0,1,2,3]]
print(emp_trust)

Q21              1         2         3         4
EMP_2010                                        
1         0.429770  0.405145  0.125830  0.029283
2         0.347746  0.435500  0.151438  0.065317
3         0.388467  0.403816  0.189112  0.018605
4         0.203923  0.341318  0.348049  0.053503
5         0.361644  0.282458  0.339722  0.016176
6         0.449050  0.411132  0.107371  0.023404


In [34]:
rel_trust = df.groupby(['D1', 'Q11B']).wgt.sum().reset_index()
rel_trust = rel_trust.pivot(columns='Q11B', index='D1', values='wgt')
#print(rel_trust)
rel_trust = rel_trust.iloc[[0,1],[0,1,2,3]]
#print(rel_trust)
stat, p, dof, ex = chi2_contingency(rel_trust)
print(p)

0.03619269476915903


In [36]:
rel_trust = (df.groupby(['D1', 'Q11B']).wgt.sum()/df.groupby('D1').wgt.sum()).reset_index()
rel_trust = rel_trust.pivot(columns='Q11B', index='D1', values='wgt')
rel_trust = rel_trust.iloc[[0,1],[0,1,2,3]]
print(rel_trust)

Q11B         1         2         3         4
D1                                          
1     0.130858  0.404853  0.235892  0.212699
2     0.087846  0.357457  0.272877  0.271947


In [35]:
rel_trust = df.groupby(['D1', 'Q21']).wgt.sum().reset_index()
rel_trust = rel_trust.pivot(columns='Q21', index='D1', values='wgt')
#print(rel_trust)
rel_trust = rel_trust.iloc[[0,1],[0,1,2,3]]
#print(rel_trust)
stat, p, dof, ex = chi2_contingency(rel_trust)
print(p)

0.20941816452882592


# Trust me, I'm from the government
#1 assess trust in government and medics (general and for health advice)
#2 compare
#3 which variables have an effect
#4 model trust vs multiple variables
#5 which specific demographics are more/less trusting
#6 discussion

In [18]:
dtree = tree.DecisionTreeClassifier(max_depth=6)

In [4]:
vars = df[['Subjective_Income', 'EMP_2010', 'AgeCategories', 'Education', 'D1', 'Q21']]

In [5]:
vars = vars.apply(pd.to_numeric, errors='coerce').dropna()
varsk = vars[(vars < 90).all(1)]
X = varsk[['Subjective_Income', 'EMP_2010', 'AgeCategories', 'Education', 'D1']]
y = varsk['Q21']
print(vars.groupby('D1').Q21.count())
print(varsk.groupby('D1').Q21.count())

D1
1     660
2     294
99     22
Name: Q21, dtype: int64
D1
1    657
2    288
Name: Q21, dtype: int64


In [19]:
dtree.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [21]:
test1 = [[3, 2, 2, 2, 1], [1, 2, 2, 2, 1], [3, 1, 2, 2, 1], [1, 1, 2, 2, 1], [3, 2, 2, 3, 1], [1, 2, 2, 3, 1]]
test2 = [[1, 2, 2, 2, 1]]
test3 = [[3, 1, 2, 2, 1]]
test4 = [[1, 1, 2, 2, 1]]
test5 = [[3, 2, 2, 3, 1]]
test6 = [[1, 2, 2, 3, 1]]
test7 = [[3, 1, 2, 3, 1]]
test8 = [[1, 1, 2, 3, 1]]
test9 = [[3, 2, 2, 2, 2]]
test10 = [[1, 2, 2, 2, 2]]
test11 = [[3, 1, 2, 2, 2]]
test12 = [[1, 1, 2, 2, 2]]
print(dtree.predict(test1))
print(dtree.predict(test2))
print(dtree.predict(test3))
print(dtree.predict(test4))
print(dtree.predict(test5))
print(dtree.predict(test6))
print(dtree.predict(test7))
print(dtree.predict(test8))
print(dtree.predict(test9))
print(dtree.predict(test10))
print(dtree.predict(test11))
print(dtree.predict(test12))
print(dtree.predict_proba(test1))
print(dtree.predict_proba(test2))
print(dtree.predict_proba(test3))
print(dtree.predict_proba(test4))
print(dtree.predict_proba(test5))
print(dtree.predict_proba(test6))
print(dtree.predict_proba(test7))
print(dtree.predict_proba(test8))
print(dtree.predict_proba(test9))
print(dtree.predict_proba(test10))
print(dtree.predict_proba(test11))
print(dtree.predict_proba(test12))

[2 1 2 1 2 1]
[1]
[2]
[1]
[2]
[1]
[2]
[1]
[2]
[2]
[2]
[2]
[[0.16666667 0.66666667 0.         0.16666667]
 [0.6        0.32       0.04       0.04      ]
 [0.36842105 0.47368421 0.10526316 0.05263158]
 [0.6        0.32       0.04       0.04      ]
 [0.16666667 0.66666667 0.         0.16666667]
 [0.45555556 0.43333333 0.08888889 0.02222222]]
[[0.6  0.32 0.04 0.04]]
[[0.36842105 0.47368421 0.10526316 0.05263158]]
[[0.6  0.32 0.04 0.04]]
[[0.16666667 0.66666667 0.         0.16666667]]
[[0.45555556 0.43333333 0.08888889 0.02222222]]
[[0.36842105 0.47368421 0.10526316 0.05263158]]
[[0.45555556 0.43333333 0.08888889 0.02222222]]
[[0.125 0.875 0.    0.   ]]
[[0.45454545 0.54545455 0.         0.        ]]
[[0.125 0.875 0.    0.   ]]
[[0.45454545 0.54545455 0.         0.        ]]


In [22]:
trust_score = [3, 2, 1, 0]
test_score = dtree.predict_proba(test1).dot(trust_score)
print(test_score)
test_score12 = dtree.predict_proba(test11).dot(trust_score)
print(test_score12)

[1.83333333 2.48       2.15789474 2.48       1.83333333 2.32222222]
[2.125]


In [35]:
inc = [1,2,3]
emp = [1,2,3,4,5,6]
age = [1,2,3]
ed = 3
rel = 2

all_demographics = []

for i in inc:
    for e in emp:
        for a in age:
            newarr = [i, e, a]
            all_demographics.append(newarr)

print(all_demographics)

[[1, 1, 1], [1, 1, 2], [1, 1, 3], [1, 2, 1], [1, 2, 2], [1, 2, 3], [1, 3, 1], [1, 3, 2], [1, 3, 3], [1, 4, 1], [1, 4, 2], [1, 4, 3], [1, 5, 1], [1, 5, 2], [1, 5, 3], [1, 6, 1], [1, 6, 2], [1, 6, 3], [2, 1, 1], [2, 1, 2], [2, 1, 3], [2, 2, 1], [2, 2, 2], [2, 2, 3], [2, 3, 1], [2, 3, 2], [2, 3, 3], [2, 4, 1], [2, 4, 2], [2, 4, 3], [2, 5, 1], [2, 5, 2], [2, 5, 3], [2, 6, 1], [2, 6, 2], [2, 6, 3], [3, 1, 1], [3, 1, 2], [3, 1, 3], [3, 2, 1], [3, 2, 2], [3, 2, 3], [3, 3, 1], [3, 3, 2], [3, 3, 3], [3, 4, 1], [3, 4, 2], [3, 4, 3], [3, 5, 1], [3, 5, 2], [3, 5, 3], [3, 6, 1], [3, 6, 2], [3, 6, 3]]


In [14]:
dotfile = open("dtree2.dot", 'w')
tree.export_graphviz(dtree, out_file = dotfile, feature_names = X.columns)
dotfile.close()

In [37]:
def trust_outcome(response):
    trust_score = np.array([3, 2, 1, 0])
    score = response[0].dot(trust_score)
    gini_imp = 0
    for i in range(4):
        gini_imp += response[0][i]**2
    return [gini_imp, score]

In [41]:
test = pd.DataFrame([0.25, 0.75, 0, 0])
print(trust_outcome(test))
test = pd.DataFrame([0.25, 0.25, 0.25, 0.25])
print(trust_outcome(test))
test = pd.DataFrame([0, 1, 0, 0])
print(trust_outcome(test))
test = pd.DataFrame([0, 0, 0.25, 0.75])
print(trust_outcome(test))

[0.625, 2.25]
[0.25, 1.5]
[1, 2]
[0.625, 0.25]


In [53]:
print(test)

      0
0  0.00
1  0.00
2  0.25
3  0.75


In [52]:
inc_1 = pd.DataFrame([0.492078, 0.380133, 0.090487, 0.029209])
inc_2 = pd.DataFrame([0.400138, 0.414273, 0.158348, 0.019950])
inc_3 = pd.DataFrame([0.254850, 0.427451, 0.242898, 0.056311])
print(trust_outcome(inc_1))
print(trust_outcome(inc_2))
print(trust_outcome(inc_3))
print(pd.DataFrame([[1,'','','','',''], [2,'','','',''], [3,'','','','']], columns=['Inc', 'Emp', 'Age', 'Ed', 'Rel', 'Score']))

[0.39568291862300004, 2.326987]
[0.357204629177, 2.187308]
[0.30983324702600007, 1.8623500000000002]
   Inc Emp Age Ed Rel Score
0    1                     
1    2                 None
2    3                 None
